In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
# !pip install plotnine
from plotnine import *
import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
import time

In [2]:
df = pd.read_csv('D:/Data/EFA_AirLine/train.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [3]:
df[df.satisfaction=='neutral or dissatisfied']=1
df[df.satisfaction=='satisfied']=0

In [4]:
df.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

Let's separate them out into numerical and categorical columns and the columns we need to drop. The column that holds the labels is `satisfaction` , we save it as `y_col`

In [5]:
drop_cols = ["Unnamed: 0" , "id"]
num_cols = ["Age" , "Flight Distance" ,"Departure Delay in Minutes" , "Arrival Delay in Minutes"]
y_col = "satisfaction"
cat_cols = list(set(df.columns).difference(set(drop_cols+num_cols)))

In [6]:
ohe = OneHotEncoder(drop="first")
cat_df = pd.DataFrame(ohe.fit_transform(df[cat_cols]).todense() , columns=ohe.get_feature_names(cat_cols))

In [7]:
mms = MinMaxScaler()
num_df = pd.DataFrame(mms.fit_transform(df[num_cols]) , columns=num_cols)

Now we join our categorical dataframe with our numerical dataframe.

In [8]:
X = cat_df.join(num_df)
y = df[y_col]

In [9]:
X = X.fillna(X.mean()) ### replacing missing values with mean

In [10]:
X_train , X_test , y_train  ,y_test = train_test_split(X[:10000],y[:10000],test_size=.33,random_state=42)

In [11]:
kmeans = KMeans(2)
kmeans.fit(X_train)

KMeans(n_clusters=2)

In [12]:
pred = kmeans.predict(X)
pred

array([1, 1, 0, ..., 1, 1, 1])

In [13]:
y

0         1
1         1
2         0
3         1
4         0
         ..
103899    1
103900    0
103901    1
103902    1
103903    1
Name: satisfaction, Length: 103904, dtype: object

In [14]:
c = 0
for i,j in zip(pred,y) :
    if i == j :
        c += 1
print(c*100/len(y))

100.0


In [15]:
logging.info('TSNE started')

2021-04-28 03:12:37,495 - TSNE started


In [ ]:
tsne = TSNE(2 , n_iter=250)
transformed_df = tsne.fit_transform(X_train)

In [ ]:
logging.info('TSNE Finished')

In [ ]:
(
    ggplot(pd.DataFrame({"c1" : transformed_df[: , 0]
                        ,"c2" : transformed_df[: , 1]
                        ,"cluster" : kmeans.labels_}) , aes(x="c1" , y="c2" , fill="cluster"))
    + geom_point(alpha=0.4 , stroke=0)
)

In [ ]:
(
    ggplot(pd.DataFrame({"c1" : transformed_df[: , 0]
                        ,"c2" : transformed_df[: , 1]
                        ,"y" : y_train}) , aes(x="c1" , y="c2" , fill="y"))
    + geom_point(alpha=0.4 , stroke=0)
)

In [ ]:
result_df = X_train.copy()
result_df["cluster"] = kmeans.labels_

In [ ]:
melt_cluster = result_df.groupby("cluster").mean().reset_index().melt(id_vars="cluster")
melt_cluster

In [ ]:
melt_cluster = melt_cluster["variable"].str.split("_" , expand=True).join(melt_cluster)
melt_cluster = melt_cluster.rename({0 : "variable_base" , 1:"response"} , axis=1)
melt_cluster